In [122]:
import numpy as np
import csv
from scipy.sparse import csc_matrix

#constants defining the dimensions of our User Rating Matrix (URM)
MAX_BID = 31 #MAX Bolt ID
MAX_TID = 11 #MAX Termékcsoport
K=2



def readUrm():
	urm = np.zeros(shape=(MAX_BID,MAX_TID), dtype=np.float32)
	with open('SVDTrain.csv', 'rb') as trainFile:
		urmReader = csv.reader(trainFile, delimiter=';')
		for row in urmReader:
			urm[int(row[0]), int(row[1])] = float(row[2])

	return csc_matrix(urm, dtype=np.float32)

In [123]:
data=readUrm()

In [124]:
data.todense()

matrix([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.66000000e+04,   2.69000000e+04,
           7.40000000e+03,   1.30000000e+03,   6.78000000e+04,
           9.00000000e+03,   4.03000000e+04,   2.40000000e+04,
           8.00000000e+03,   2.41800000e+05],
        [  0.00000000e+00,   4.73000000e+04,   6.68000000e+04,
           3.47000000e+04,   8.20000000e+03,   2.37500000e+05,
           4.26000000e+04,   1.60900000e+05,   2.92000000e+04,
           3.60000000e+04,   4.56100000e+05],
        [  0.00000000e+00,   3.17000000e+04,   6.60000000e+03,
           2.70000000e+03,   0.00000000e+00,   4.94000000e+04,
           5.50000000e+03,   2.21000000e+04,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   5.30000000e+0

In [125]:
def readUsersTest():
	uTest = dict()
	with open("SVDBoltok.csv", 'rb') as testFile:
		testReader = csv.reader(testFile, delimiter=';')
		i=0
		for row in testReader:
			uTest[int(row[0])] = list()

	return uTest

In [126]:
def getMoviesSeen():
	moviesSeen = dict()
	with open("SVDTrain.csv", 'rb') as trainFile:
		urmReader = csv.reader(trainFile, delimiter=';')
		for row in urmReader:
			try:
				moviesSeen[int(row[0])].append(int(row[1]))
			except:
				moviesSeen[int(row[0])] = list()
				moviesSeen[int(row[0])].append(int(row[1]))

	return moviesSeen

In [127]:
import math as mt
import csv
from sparsesvd import sparsesvd

def computeSVD(urm, K):
	U, s, Vt = sparsesvd(urm, K)

	dim = (len(s), len(s))
	S = np.zeros(dim, dtype=np.float32)
	for i in range(0, len(s)):
		S[i,i] = mt.sqrt(s[i])

	U = csc_matrix(np.transpose(U), dtype=np.float32)
	S = csc_matrix(S, dtype=np.float32)
	Vt = csc_matrix(Vt, dtype=np.float32)

	return U, S, Vt	

In [128]:
from scipy.sparse.linalg import * #used for matrix multiplication

def computeEstimatedRatings2(urm, U, S, Vt, uTest, moviesSeen, K, test):
	rightTerm = S*Vt 

	estimatedRatings = np.zeros(shape=(MAX_BID, MAX_TID), dtype=np.float16)
	for userTest in uTest:
		prod = U[userTest, :]*rightTerm

		#we convert the vector to dense format in order to get the indices of the movies with the best estimated ratings 
		estimatedRatings[userTest, :] = prod.todense()
		recom = (-estimatedRatings[userTest, :]).argsort()
		for r in recom:
			if r not in moviesSeen[userTest]:
				uTest[userTest].append(r)


	return uTest

In [129]:
from scipy.sparse.linalg import * #used for matrix multiplication

def computeEstimatedRatings(urm, U, S, Vt, uTest, moviesSeen, K, test):
	rightTerm = S*Vt 

	estimatedRatings = np.zeros(shape=(MAX_BID, MAX_TID), dtype=np.float16)
	for userTest in uTest:
		prod = U[userTest, :]*rightTerm

		#we convert the vector to dense format in order to get the indices of the movies with the best estimated ratings 
		estimatedRatings[userTest, :] = prod.todense()
		recom = (-estimatedRatings[userTest, :])
		for r in recom:
			if r not in moviesSeen[userTest]:
				uTest[userTest].append(r)

	return uTest

In [146]:
def main():
    #értékek
    K = 20
    urm = readUrm()
    U, S, Vt = computeSVD(urm,K)
    uTest = readUsersTest()
    moviesSeen = getMoviesSeen()
    uTest = computeEstimatedRatings(urm, U, S, Vt, uTest, moviesSeen, K, True)
    #indexek
    Ux, Sx, Vtx = computeSVD(urm,K)
    uTestx = readUsersTest()
    moviesSeen = getMoviesSeen()
    uTestx = computeEstimatedRatings2(urm, Ux, Sx, Vtx, uTestx, moviesSeen, K, True)

    for key in uTest.keys():
        for i in uTestx[key]:
            if(i!=0):
                print("Bolt: %d , Termék család:%d érték:%.2f" % (key,i,uTest[key][i]))

In [147]:
main()

Bolt: 3 , Termék család:10 érték:17.31
Bolt: 3 , Termék család:9 érték:18.09
Bolt: 3 , Termék család:8 érték:26.62
Bolt: 4 , Termék család:1 érték:16.23
Bolt: 30 , Termék család:8 érték:30.12
Bolt: 30 , Termék család:9 érték:67.62
Bolt: 30 , Termék család:7 érték:111.75
